## load libray

In [1]:
import os
import sys
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from queue import Queue

## sort function

In [2]:
def sort_items(a):
    path="./images5"
    return int(a[len(path)+1:len(a)-4])

## Load images

In [3]:
path="./images5"
images = glob.glob(path+"/*.png")
images.sort(key=sort_items)
#a=sorted(images, key=sort_items)
#print(images)img

## Thresholding customize funtion

In [4]:
def thrsh1(img,t):
    inpt=img.copy()
    for i in range (len(inpt)):
        for j in range (len(inpt[i])):
            if inpt[i][j]>t:
                inpt[i][j]=t
    ret, thresh1 = cv2.threshold(inpt, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    cv2.imwrite("thresh1"+str(int(t))+".png",thresh1)
    plt.imshow(thresh1)
    return ret

def thrsh2(img,t):
    inpt=img.copy()
    for i in range(len(inpt)):
        for j in range (len(inpt[i])):
            if inpt[i][j]<t:
                inpt[i][j]=t
    ret, thresh1 = cv2.threshold(inpt, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    cv2.imwrite("thresh2"+str(int(t))+".png",thresh1)
    plt.imshow(thresh1)
    return ret,thresh1

def setpx(img,t1,t2,t3):
    inpt=img.copy()
    inpt1 = img.copy()
    inpt2 = img.copy()
    inpt3 = img.copy()
    for i in range(len(inpt)):
        for j in range (len(inpt[i])):
            if (inpt[i][j]<t1):
                inpt[i][j]=0
            elif(inpt[i][j]<t2):
                inpt[i][j]=t1
            elif (inpt[i][j]<t3):
                inpt[i][j]=t2
            else:
                inpt[i][j]=t3
    cv2.imwrite("Final.png",inpt)
    plt.imshow(inpt)
    return inpt

def writepx(img,t1,t2,t3):
    inpt=img.copy()
    inpt1 = img.copy()
    inpt2 = img.copy()
    inpt3 = img.copy()
    for i in range(len(inpt)):
        for j in range (len(inpt[i])):
            if (inpt1[i][j]==t1):
                inpt1[i][j]=t1
            else:
                inpt1[i][j]=0
            if(inpt2[i][j]==t2):
                inpt2[i][j]=t2
            else:
                inpt2[i][j]=0
            if (inpt3[i][j]==t3):
                inpt3[i][j]=t3
            else:
                inpt3[i][j]=0
                
    cv2.imwrite("inpt1.png",inpt1)
    cv2.imwrite("inpt2.png",inpt2)
    cv2.imwrite("inpt3.png",inpt3)

SyntaxError: invalid syntax (<ipython-input-4-3d09f34b8cc1>, line 35)

## im_show function

In [ ]:
def im_show(img):
    cv2.imshow("image", img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows() 

## Crop image using mask funtion

In [ ]:
def crop(img):
    image = cv2.imread(img)
    #imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = np.zeros(image.shape[:2], dtype="uint8")
    pts = np.array([[1350,590], [1700,590], [1700,720], [1350,720]])
    color = 255
    cv2.fillPoly(mask, [pts], color)
    masked = cv2.bitwise_and(image, image, mask=mask)
    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    croped = image[y:y+h, x:x+w].copy()
    print(croped.shape, image.shape)
    return croped, image

## Calculate Distance

In [ ]:
# cooridinates is the array of touples
def distance_matrix_of_bars(coordinates):
    a=[]
    for i in range (len(coordinates)):
        b=[]
        for x in coordinates:
            b.append(cord_distance([coordinates[i][0],coordinates[i][1]],
            [x[0],x[1]]))
        a.append(b)
    return a

def cord_distance(cord1, cord2):
    point1 = np.array(cord1)
    point2 = np.array(cord2)
    # finding sum of squares
    sum_sq = np.sum(np.square(point1 - point2))
    # Doing squareroot and
    # printing Euclidean distance
    return (np.sqrt(sum_sq))


## Counting Algo

In [ ]:
def castx(x):
    return x[0]

tlr=50 #tollerance threshhold for count
x_max=250  # will not consider bar whose cordinate x value greater than x_max
def count_bar(buffer_cont_stats,avg):
    count=0
    distance_dict = {}
    processing_frame_no = int(buffer_cont_stats.qsize()/2)
    if(buffer_cont_stats.full()):
        buffer_stats_array=buffer_cont_stats.queue
        for i in range(len(buffer_stats_array)):
            for j in range(len(buffer_stats_array[i])):
                if(buffer_stats_array[i][j][5]==-1):
                    buffer_stats_array[i][j][5]=int(buffer_stats_array[i][j][4]/avg)                    
            buffer_stats_array[i].sort(key=castx)
            distance_dict[i] = distance_matrix_of_bars(buffer_stats_array[i])        
        m1=np.array(distance_dict[0])
        m2=np.array(distance_dict[processing_frame_no])
        if(np.array_equal(m1,m2)):
            return count;
        else:
            if (len(buffer_stats_array[processing_frame_no])==
                len(buffer_stats_array[0])):
                initial_frame=buffer_stats_array[0]
                processing_frame=buffer_stats_array[processing_frame_no]
                num=0
                for i in range(len(initial_frame)):
                    if(initial_frame[i][4]==processing_frame[i][4]):
                        num+=1
                    else:                        
                        if(initial_frame[i][0]>x_max):
                            break
                        num=0
                        dif_area = initial_frame[i][4]-processing_frame[i][4]
                        if(abs(dif_area-avg)<tlr):                            
                            count += 1
                            for j in range(processing_frame_no):
                                buffer_stats_array[j][i][4]=processing_frame[i][4]
                    if(num>5):
                        break
            else:
                initial_frame=buffer_stats_array[0]
                processing_frame=buffer_stats_array[processing_frame_no]
                num=0
                for i in range(len(initial_frame)):
                    if(initial_frame[i][4]==processing_frame[i][4]):
                        num+=1
                    else:                        
                        if(initial_frame[i][0]>x_max):
                            break
                        num=0
                        dif_area = initial_frame[i][4]-processing_frame[i][4]
                        if(abs(dif_area-avg)<tlr):                            
                            count += 1
                            for j in range(processing_frame_no):
                                buffer_stats_array[j][i][4]=processing_frame[i][4]
                    if(num>5):
                        break
    return count

## Supporting algo find contours

In [ ]:
#call with croped image
def threshing(img):
    imageGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    median_img = cv2.medianBlur(imageGray, 5) # check with gaussian filter
    ret2, thresh1 = cv2.threshold(median_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    ret1=thrsh1(median_img,ret2)
    ret3,tr3=thrsh2(median_img,ret2)
    ret3,tr3=thrsh2(median_img,ret3)
    ret3,tr3=thrsh2(median_img,ret3)
    ret3,tr3=thrsh2(median_img,ret3)
    ret3,tr3=thrsh2(median_img,ret3)
    ret3,tr3=thrsh2(median_img,ret3)
    return ret3,tr3
# lower_bound = 180, upper_bound = 450
#  contour_distance = {}
def find_contours(img, frame_no, lower_bound, upper_bound,
     buffer_cont_stats):
    #contours, hierarchy = cv2.findContours(tr3, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    ret3,tr3=threshing(img)
    contours,hierarchy = cv2.findContours(tr3, 1, 2)
    avg_area = 0
    large_area = 0
    count = 0
    a=[]
    for i in range(len(contours)):
        cnt = contours[i]        
        rect = cv2.boundingRect(cnt)
        x,y,w,h = rect
        area = cv2.contourArea(cnt)
        if(area>lower_bound and area<upper_bound):
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            avg_area += area
            count +=1
            a.append([cx,cy,w,h,area,-1])
        if(area >upper_bound):
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            large_area += area
            a.append([cx,cy,w,h,area,-1])
        #total_area += area
        #perimeter = cv2.arcLength(cnt,True)
        #print(cx,cy,area,perimeter)
    if count>0:
        avg=(avg_area/count)
    else:
        avg=350
    if(buffer_cont_stats.full()):
        buffer_cont_stats.get()
    buffer_cont_stats.put(a)
    return avg

## Main algo

In [ ]:
x,y = 1000,10
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2buffer_stats_array
color=(0,0,0)
count=0
out_path="./out5/"
buffer_size = 9
buffer_cont_stats = Queue(maxsize=buffer_size)
lower_bound, upper_bound = 180, 450
if(frame_no<140):
    continue
croped, original = crop(img)
avg = find_contours(croped, frame_no, lower_bound, upper_bound, buffer_cont_stats)
count+=count_bar(buffer_cont_stats,avg)
print(count)
    # x1=x+croped.shape[1]+200
    # y1= y +20
    # red_box = cv2.rectangle(original, (x1, y1), (x1+300, y1+100), (255, 255, 255), -1)
    # original[y:y+croped.shape[0],x:x+croped.shape[1]] = croped
    # org=(x1+5,y1+60)
    # #im_show(croped)
    # image = cv2.putText(original, str(count), org, font, fontScale, color, thickness, cv2.LINE_AA)
    # cv2.imwrite(out_path+img[len(path)+1:],image)
    # #im_show(original)
    

1.png
2.png
3.png
4.png
5.png
6.png
7.png
8.png
9.png
10.png
11.png
12.png
13.png
14.png
15.png
16.png
17.png
18.png
19.png
20.png
21.png
22.png
23.png
24.png
25.png
26.png
27.png
28.png
29.png
30.png
31.png
32.png
33.png
34.png
35.png
36.png
37.png
38.png
39.png
40.png
41.png
42.png
43.png
44.png
45.png
46.png
47.png
48.png
49.png
50.png
51.png
52.png
53.png
54.png
55.png
56.png
57.png
58.png
59.png
60.png
61.png
62.png
63.png
64.png
65.png
66.png
67.png
68.png
69.png
70.png
71.png
72.png
73.png
74.png
75.png
76.png
77.png
78.png
79.png
80.png
81.png
82.png
83.png
84.png
85.png
86.png
87.png
88.png
89.png
90.png
91.png
92.png
93.png
94.png
95.png
96.png
97.png
98.png
99.png
100.png
101.png
102.png
103.png
104.png
105.png
106.png
107.png
108.png
109.png
110.png
111.png
112.png
113.png
114.png
115.png
116.png
117.png
118.png
119.png
120.png
121.png
122.png
123.png
124.png
125.png
126.png
127.png
128.png
129.png
130.png
131.png
132.png
133.png
134.png
135.png
136.png
137.png
138.png
139.

AttributeError: module 'cv2' has no attribute 'thresholcount_bard'